task name: Implement RandomizedPcaSampleWithOptions
task description: Implement the provided C# code RandomizedPcaSampleWithOptions as a dotnet interactive notebook

In [ ]:
#r "nuget:Microsoft.ML"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
// Example with 3 feature values. A training data set is a collection of such examples.
private class DataPoint
{
    [VectorType(3)]
    public float[] Features { get; set; }
}

var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {0, 2, 3} },
    new DataPoint(){ Features = new float[3] {0, 2, 4} },
    new DataPoint(){ Features = new float[3] {0, 2, 1} },
    new DataPoint(){ Features = new float[3] {0, 2, 2} },
    new DataPoint(){ Features = new float[3] {0, 2, 3} },
    new DataPoint(){ Features = new float[3] {0, 2, 4} },
    new DataPoint(){ Features = new float[3] {1, 0, 0} }
};

var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var options = new Microsoft.ML.Trainers.RandomizedPcaTrainer.Options()
{
    FeatureColumnName = nameof(DataPoint.Features),
    Rank = 1,
    Seed = 10,
};

var pipeline = mlContext.AnomalyDetection.Trainers.RandomizedPca(options);

In [ ]:
var model = pipeline.Fit(data);

In [ ]:
var transformed = model.Transform(data);

In [ ]:
private class Result
{
    // Outlier gets true while inlier has false.
    public bool PredictedLabel { get; set; }
    // Inlier gets smaller score. Score is between 0 and 1.
    public float Score { get; set; }
}

var results = mlContext.Data.CreateEnumerable<Result>(transformed, reuseRowObject: false).ToList();

In [ ]:
for (int i = 0; i < samples.Count; ++i)
{
    var result = results[i];
    var featuresInText = string.Join(',', samples[i].Features);

    if (result.PredictedLabel)
        Console.WriteLine($"The {i}-th example with features [{featuresInText}] is an outlier with a score of being outlier {result.Score}");
    else
        Console.WriteLine($"The {i}-th example with features [{featuresInText}] is an inlier with a score of being outlier {result.Score}");
}